In [18]:
import numpy as np
from math import gcd, lcm
from functools import reduce

graph = {}

with open("input.txt") as input:
    directions = input.readline().rstrip()
    directions = directions.replace("R", "1").replace("L", "0")
    input.readline()  # skip empty line

    for line in input:
        node, sides = line.rstrip().split(" = ")
        left, right = sides[1:-1].split(", ")
        graph[node] = (left, right)

In [4]:
# z_steps_table[state][steps % len(directions)]
# gives the number of steps to the next Z state and the next state:
# (steps, next_state)
z_steps_table = {}
def build_z_steps_table():
    for initial_state in graph.keys():
        if initial_state[-1] != "Z" and initial_state[-1] != "A":
            continue
        z_steps_table[initial_state] = [0] * len(graph)
        for offset in range(len(directions)):
            steps = 0
            state = initial_state
            while steps == 0 or state[-1] != "Z":
                dir2go = int(directions[(steps + offset) % len(directions)])
                steps += 1
                state = graph[state][dir2go]
                assert steps < 1_000_000
            z_steps_table[initial_state][offset] = (steps, state)

build_z_steps_table()

In [8]:
starting_list = [x for x in graph if x[-1] == "A"]
steps = [z_steps_table[z][0][0] for z in starting_list]
states= [z_steps_table[z][0][1] for z in starting_list]
assert all((z>0 for z in steps))
diff = [0] * len(steps)
for rounds in range(100):
    i = np.argmin(steps)
    new_step_tuple = z_steps_table[states[i]][steps[i] % len(directions)]
    steps[i] += new_step_tuple[0]
    if diff[i] != new_step_tuple[0]:
        print('diff change:', i, new_step_tuple[0])
    diff[i] = new_step_tuple[0]
    states[i] = new_step_tuple[1]


diff change: 2 11309
diff change: 0 13939
diff change: 5 15517
diff change: 1 17621
diff change: 4 19199
diff change: 3 20777


Note that we got a constant step per starting state, which means that they will all overlap at the LCM(diff)

In [16]:
# Solution to part 2
reduce(lcm, diff, 1)

13663968099527

## Some fun

Let's find the GCD of them:

In [21]:
g = reduce(gcd, diff, diff[0])
g

263

In [27]:
diff_g = np.array(diff) // g
diff_g

array([53, 67, 43, 79, 73, 59])

In [29]:
# Now they are co-prime, so let's multiply and get the same solution

np.prod(diff_g) * g

13663968099527